## **Connecting to Google Drive**

In [ ]:
# Import the drive module from google.colab to mount Google Drive
from google.colab import drive

# Mount Google Drive to the Colab environment
# This will prompt you to authorize access to your Google Drive account
drive.mount('/content/drive')

# Define the path to the training dataset located in Google Drive
data_root = '/content/drive/MyDrive/animal/Train'

# Define the path to the test dataset located in Google Drive
data_test = '/content/drive/MyDrive/animal/Test'


Mounted at /content/drive


## **Python Liabrires**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import cv2
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support
from sklearn.utils import class_weight

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models , regularizers, optimizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import InceptionV3

## **Data Preprocessing**

In [ ]:
# Define image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 64
Training_data_dir = str(data_root)
Testing_data_dir = str(data_test)

# Data Augmentation for the training set
# The ImageDataGenerator will apply random transformations to the training images
# This helps in reducing overfitting and improving the model's generalization
train_datagen = ImageDataGenerator(
    rotation_range=10,
    rescale=1.0/255,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1.0/255,validation_split=.20)
test_datagen = ImageDataGenerator(rescale=1.0/255)


# Data generator for the training set
train_generator = train_datagen.flow_from_directory(
    Training_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    subset="training",
    class_mode='categorical',
    shuffle=True)

# Data generator for the validation set
validation_generator = validation_datagen.flow_from_directory(
    Training_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    subset="validation",
    class_mode='categorical',
    shuffle=True)

# Data generator for the test set
test_generator = test_datagen.flow_from_directory(
    Testing_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

# Map class indices to their respective class labels
labels = {value: key for key, value in train_generator.class_indices.items()}

# Print the label mappings to understand which index corresponds to which class
print("Label Mappings for classes present in the training and validation datasets\n")
for key, value in labels.items():
    print(f"{key} : {value}")


Found 5964 images belonging to 6 classes.
Found 1191 images belonging to 6 classes.
Found 759 images belonging to 6 classes.
Label Mappings for classes present in the training and validation datasets

0 : Cheetah
1 : Elephant
2 : Giraffe
3 : Lion
4 : Rhino
5 : Zebra


 This cell sets up the image preprocessing and data augmentation pipeline. It configures ImageDataGenerator to apply transformations to training images (e.g., rotation, scaling, flipping) for better generalization. It then creates data generators for training, validation, and testing datasets, allowing efficient loading and preprocessing.

## **Class Weight**

In [ ]:
# Calculate class weights to handle class imbalance
# This ensures that classes with fewer samples are given higher weight during training
# 'balanced' mode uses the formula: n_samples / (n_classes * np.bincount(y))
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)

# Convert class weights to a dictionary format required by Keras
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}
print("Class Weights:", class_weights_dict)

Class Weights: {0: 1.1721698113207548, 1: 1.0463157894736843, 2: 1.004040404040404, 3: 0.9851337958374629, 4: 0.9019963702359347, 5: 0.9333333333333333}


## **Cross Validation**

In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Create stratified K-fold splits
data_generator = train_datagen.flow_from_directory(
    Training_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

# Retrieve class indices for the training dataset
labels = data_generator.classes
classes = list(data_generator.class_indices.keys())

num_folds = 5
# Perform K-Fold Cross Validation
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
fold_no = 1
acc_per_fold = []
loss_per_fold = []

Found 5964 images belonging to 6 classes.


### **Base CNN Model**

In [ ]:
fold_no = 1
for train_index, val_index in skf.split(np.zeros(len(labels)), labels):
    print(f"Fold - {fold_no}")
    def create_model():
        model = Sequential([
            Conv2D(filters=64, kernel_size=(5, 5), padding='valid', input_shape=(img_width, img_height, 3)),
            Activation('relu'),
            MaxPooling2D(pool_size=(2, 2)),
            BatchNormalization(),

            Conv2D(filters=128, kernel_size=(3, 3), padding='valid', kernel_regularizer=l2(0.00005)),
            Activation('relu'),
            MaxPooling2D(pool_size=(2, 2)),
            BatchNormalization(),
            Flatten(),

            Dense(units=2048, activation='relu'),
            Dropout(0.5),
            Dense(units=6, activation='softmax')
        ])

        return model

    cnn_model = create_model()

    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1), patience=5)
    optimizer = Adam(learning_rate=0.001)
    cnn_model.compile(optimizer=optimizer, loss=CategoricalCrossentropy(), metrics=['accuracy'])


    history = cnn_model.fit(
        train_generator,
        epochs=30,
        validation_data=validation_generator,
        verbose=2,
        callbacks=[reduce_lr],
        class_weight=class_weights_dict
        )

    # Evaluate the model on the validation data for the current fold
    val_loss, val_acc = cnn_model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
    print(f"Fold - {fold_no}")
    print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")
    print("--------------------------------------------------------------------------------------------------------")
    acc_per_fold.append(val_acc)
    loss_per_fold.append(val_loss)

    fold_no += 1

# Print the average accuracy and loss across all folds
print("Average Validation Accuracy:", np.mean(acc_per_fold))
print("Average Validation Loss:", np.mean(loss_per_fold))

# Evaluate the final model on the test set
test_loss, test_acc = cnn_model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_acc}")

Fold - 1
Epoch 1/30
103/103 - 2363s - 23s/step - accuracy: 0.4126 - loss: 11.2349 - val_accuracy: 0.1716 - val_loss: 3.6646 - learning_rate: 0.0010
Epoch 2/30
103/103 - 74s - 721ms/step - accuracy: 0.4428 - loss: 3.9942 - val_accuracy: 0.2578 - val_loss: 2.4647 - learning_rate: 0.0010
Epoch 3/30
103/103 - 81s - 789ms/step - accuracy: 0.4821 - loss: 2.0083 - val_accuracy: 0.1960 - val_loss: 2.1647 - learning_rate: 0.0010
Epoch 4/30
103/103 - 78s - 762ms/step - accuracy: 0.5372 - loss: 1.3340 - val_accuracy: 0.3425 - val_loss: 1.5914 - learning_rate: 0.0010
Epoch 5/30
103/103 - 81s - 788ms/step - accuracy: 0.5741 - loss: 1.2035 - val_accuracy: 0.4661 - val_loss: 1.4004 - learning_rate: 0.0010
Epoch 6/30
103/103 - 83s - 805ms/step - accuracy: 0.6037 - loss: 1.0857 - val_accuracy: 0.6476 - val_loss: 1.0295 - learning_rate: 0.0010
Epoch 7/30
103/103 - 82s - 801ms/step - accuracy: 0.6218 - loss: 1.0428 - val_accuracy: 0.6850 - val_loss: 0.8478 - learning_rate: 0.0010
Epoch 8/30
103/103 - 80s

### **VGG19**

In [ ]:
fold_no = 1
for train_index, val_index in skf.split(np.zeros(len(labels)), labels):
    print(f"Fold - {fold_no}")
    # Load VGG19 model pre-trained on ImageNet, without the top classification layer
    base_model = VGG19(input_shape=(150, 150, 3), include_top=False, weights='imagenet')

    # Freeze the base model
    base_model.trainable = False

    # Add custom layers on top
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    predictions = layers.Dense(train_generator.num_classes, activation='softmax')(x)

    # Define the model
    VGG19_model = models.Model(inputs=base_model.input, outputs=predictions)

    Early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    optimizer = Adam(learning_rate=0.001)
    VGG19_model.compile(optimizer=optimizer, loss=CategoricalCrossentropy(), metrics=['accuracy'])


    # Train the model with class weights
    VGG19_history = VGG19_model.fit(
        train_generator,
        epochs=40,
        validation_data=validation_generator,
        verbose=2,
        callbacks=[Early_stopping],
        class_weight=class_weights_dict
    )

    # Evaluate the model on the validation data for the current fold
    val_loss, val_acc = VGG19_model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
    print(f"Fold - {fold_no}")
    print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")
    print("-----------------------------------------------------------------------------------------------------------------------------")
    acc_per_fold.append(val_acc)
    loss_per_fold.append(val_loss)

    fold_no += 1

# Print the average accuracy and loss across all folds
print("Average Validation Accuracy:", np.mean(acc_per_fold))
print("Average Validation Loss:", np.mean(loss_per_fold))

# Evaluate the final model on the test set
test_loss, test_acc = VGG19_model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_acc}")

Fold - 1
Epoch 1/40
103/103 - 58s - 560ms/step - accuracy: 0.5510 - loss: 1.1979 - val_accuracy: 0.7452 - val_loss: 0.7303
Epoch 2/40
103/103 - 54s - 525ms/step - accuracy: 0.6853 - loss: 0.8592 - val_accuracy: 0.7941 - val_loss: 0.6087
Epoch 3/40
103/103 - 54s - 524ms/step - accuracy: 0.7197 - loss: 0.7699 - val_accuracy: 0.8078 - val_loss: 0.5535
Epoch 4/40
103/103 - 54s - 528ms/step - accuracy: 0.7322 - loss: 0.7278 - val_accuracy: 0.8146 - val_loss: 0.5310
Epoch 5/40
103/103 - 54s - 523ms/step - accuracy: 0.7552 - loss: 0.6842 - val_accuracy: 0.8268 - val_loss: 0.4849
Epoch 6/40
103/103 - 53s - 519ms/step - accuracy: 0.7505 - loss: 0.6681 - val_accuracy: 0.8215 - val_loss: 0.4994
Epoch 7/40
103/103 - 53s - 517ms/step - accuracy: 0.7584 - loss: 0.6466 - val_accuracy: 0.8337 - val_loss: 0.4508
Epoch 8/40
103/103 - 53s - 513ms/step - accuracy: 0.7633 - loss: 0.6372 - val_accuracy: 0.8368 - val_loss: 0.4560
Epoch 9/40
103/103 - 53s - 514ms/step - accuracy: 0.7765 - loss: 0.6161 - val_a

### **ResNet50**

In [ ]:
fold_no = 1
for train_index, val_index in skf.split(np.zeros(len(labels)), labels):
    print(f"Fold - {fold_no}")
    # Load MobileNetV2 model pre-trained on ImageNet, without the top classification layer
    base_model = ResNet50(input_shape=(150, 150, 3), include_top=False, weights='imagenet')

    # Unfreeze some layers of the base model
    base_model.trainable = True
    fine_tune_at = 100

    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    # Add custom layers on top
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(2048, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Dropout(0.5)(x)
    predictions = layers.Dense(train_generator.num_classes, activation='softmax')(x)

    # Define the model
    ResNet50_model = models.Model(inputs=base_model.input, outputs=predictions)

    # Train the model with early stopping and more epochs
    Early_stopping = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
    optimizer = Adam(learning_rate=0.001)
    ResNet50_model.compile(optimizer=optimizer, loss=CategoricalCrossentropy(), metrics=['accuracy'])


    ResNet50_history = ResNet50_model.fit(
        train_generator,
        epochs=40,
        validation_data=validation_generator,
        verbose=2,
        class_weight=class_weights_dict
        )


    # Evaluate the model on the validation data for the current fold
    val_loss, val_acc = ResNet50_model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
    print(f"Fold - {fold_no}")
    print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")
    print("-----------------------------------------------------------------------------------------------------------------------------")
    acc_per_fold.append(val_acc)
    loss_per_fold.append(val_loss)

    fold_no += 1

# Print the average accuracy and loss across all folds
print("Average Validation Accuracy:", np.mean(acc_per_fold))
print("Average Validation Loss:", np.mean(loss_per_fold))

# Evaluate the final model on the test set
test_loss, test_acc = ResNet50_model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_acc}")

Fold - 1
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/40
103/103 - 133s - 1s/step - accuracy: 0.2090 - loss: 9.8150 - val_accuracy: 0.1777 - val_loss: 58.2887
Epoch 2/40
103/103 - 79s - 768ms/step - accuracy: 0.3287 - loss: 1.8534 - val_accuracy: 0.1800 - val_loss: 24.7086
Epoch 3/40
103/103 - 78s - 760ms/step - accuracy: 0.3517 - loss: 1.5247 - val_accuracy: 0.3379 - val_loss: 9.4474
Epoch 4/40
103/103 - 79s - 766ms/step - accuracy: 0.3787 - loss: 1.4491 - val_accuracy: 0.2647 - val_loss: 7.5336
Epoch 5/40
103/103 - 73s - 710ms/step - accuracy: 0.4154 - loss: 1.3856 - val_accuracy: 0.1831 - val_loss: 2.8030
Epoch 6/40
103/103 - 75s - 725ms/step - accuracy: 0.4111 - loss: 1.3943 - val_accuracy: 0.3120 - val_loss: 3.4445
Epoch 7/40
103/103 - 80s - 772ms/step - accuracy: 0.4355 - loss: 1.3376 - val_accuracy: 0.3219 - val_loss: 2.4457
Epoch 8/40
103/103 - 81s - 782ms/step - accuracy: 0.4489 - loss: 1.3336 - val_accuracy: 0.2616 - val_loss: 3.3122
Epoch 9/40
103/103 - 78s - 7

### **MobileNetV2**

In [ ]:
fold_no = 1
for train_index, val_index in skf.split(np.zeros(len(labels)), labels):
    # Load MobileNetV2 model pre-trained on ImageNet, without the top classification layer
    base_model = MobileNetV2(input_shape=(150, 150, 3), include_top=False, weights='imagenet')

    # Unfreeze some layers of the base model
    base_model.trainable = True
    fine_tune_at = 100

    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    # Add custom layers on top
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(4096, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(2048, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Dropout(0.5)(x)
    predictions = layers.Dense(train_generator.num_classes, activation='softmax')(x)

    # Define the model
    MobileNetV2_model = models.Model(inputs=base_model.input, outputs=predictions)

    # Compile the model with a lower learning rate
    optimizer = optimizers.Adam(learning_rate=0.0001)
    MobileNetV2_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model with early stopping and more epochs
    Early_stopping = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)

    MobileNetV2_history_1 = MobileNetV2_model.fit(
        train_generator,
        epochs=25,
        validation_data=validation_generator,
        verbose=2,
        callbacks=[Early_stopping],
        class_weight=class_weights_dict
    )

    # Evaluate the model on the validation data for the current fold
    val_loss, val_acc = MobileNetV2_model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
    print(f"Fold - {fold_no}")
    print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")
    print("-----------------------------------------------------------------------------------------------------------------------------")
    acc_per_fold.append(val_acc)
    loss_per_fold.append(val_loss)

    fold_no += 1

# Print the average accuracy and loss across all folds
print("Average Validation Accuracy:", np.mean(acc_per_fold))
print("Average Validation Loss:", np.mean(loss_per_fold))

# Evaluate the final model on the test set
test_loss, test_acc = MobileNetV2_model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_acc}")

Epoch 1/25
103/103 - 78s - 757ms/step - accuracy: 0.8102 - loss: 40.0568 - val_accuracy: 0.9336 - val_loss: 32.2496
Epoch 2/25
103/103 - 54s - 520ms/step - accuracy: 0.9350 - loss: 25.9910 - val_accuracy: 0.9672 - val_loss: 20.2621
Epoch 3/25
103/103 - 53s - 515ms/step - accuracy: 0.9505 - loss: 16.1049 - val_accuracy: 0.9847 - val_loss: 12.3040
Epoch 4/25
103/103 - 54s - 521ms/step - accuracy: 0.9615 - loss: 9.7169 - val_accuracy: 0.9703 - val_loss: 7.3886
Epoch 5/25
103/103 - 53s - 518ms/step - accuracy: 0.9695 - loss: 5.7659 - val_accuracy: 0.9603 - val_loss: 4.4075
Epoch 6/25
103/103 - 54s - 521ms/step - accuracy: 0.9744 - loss: 3.3960 - val_accuracy: 0.9847 - val_loss: 2.5292
Epoch 7/25
103/103 - 54s - 528ms/step - accuracy: 0.9819 - loss: 1.9810 - val_accuracy: 0.9832 - val_loss: 1.4893
Epoch 8/25
103/103 - 53s - 516ms/step - accuracy: 0.9819 - loss: 1.1694 - val_accuracy: 0.9901 - val_loss: 0.8656
Epoch 9/25
103/103 - 54s - 522ms/step - accuracy: 0.9849 - loss: 0.7004 - val_accu

### **InceptionV3**

In [ ]:
fold_no = 1
for train_index, val_index in skf.split(np.zeros(len(labels)), labels):
    print(f"Fold - {fold_no}")
    # Load InceptionV3 model pre-trained on ImageNet, without the top classification layer
    base_model = InceptionV3(input_shape=(150, 150, 3), include_top=False, weights='imagenet')

    # Unfreeze some layers of the base model
    base_model.trainable = True
    fine_tune_at = 100

    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    # Add custom layers on top
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(2048, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Dropout(0.5)(x)
    predictions = layers.Dense(train_generator.num_classes, activation='softmax')(x)


    # Define the model
    InceptionV3_model = models.Model(inputs=base_model.input, outputs=predictions)


    Early_stopping = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
    optimizer = Adam(learning_rate=0.001)
    InceptionV3_model.compile(optimizer=optimizer, loss=CategoricalCrossentropy(), metrics=['accuracy'])


    InceptionV3_history = InceptionV3_model.fit(
        train_generator,
        epochs=20,
        validation_data=validation_generator,
        verbose=2,
        callbacks=[Early_stopping],
        class_weight=class_weights_dict
    )

    # Evaluate the model on the validation data for the current fold
    val_loss, val_acc = InceptionV3_model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
    print(f"Fold - {fold_no}")
    print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")
    print("-----------------------------------------------------------------------------------------------------------------------------")
    acc_per_fold.append(val_acc)
    loss_per_fold.append(val_loss)

    fold_no += 1

# Print the average accuracy and loss across all folds
print("Average Validation Accuracy:", np.mean(acc_per_fold))
print("Average Validation Loss:", np.mean(loss_per_fold))

# Evaluate the final model on the test set
test_loss, test_acc = InceptionV3_model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_acc}")

Fold - 1
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Epoch 1/20
103/103 - 143s - 1s/step - accuracy: 0.8201 - loss: 11.0074 - val_accuracy: 0.6407 - val_loss: 11.4605
Epoch 2/20
103/103 - 53s - 511ms/step - accuracy: 0.9187 - loss: 1.0904 - val_accuracy: 0.9352 - val_loss: 0.7907
Epoch 3/20
103/103 - 52s - 506ms/step - accuracy: 0.9411 - loss: 0.4759 - val_accuracy: 0.8680 - val_loss: 0.7005
Epoch 4/20
103/103 - 52s - 508ms/step - accuracy: 0.9516 - loss: 0.3030 - val_accuracy: 0.9184 - val_loss: 0.3146
Epoch 5/20
103/103 - 53s - 512ms/step - accuracy: 0.9563 - loss: 0.2689 - val_accuracy: 0.9725 - val_loss: 0.2070
Epoch 6/20
103/103 - 52s - 504ms/step - accuracy: 0.9669 - loss: 0.2126 - val_accuracy: 0.6018 - val_loss: 7.1277
Epoch 7/20
103/103 - 53s - 511ms/step - accuracy: 0.9656 - loss: 0.2275 - val_accuracy: 0.9901 - val_loss: 0.1379
Epoch 8/20
103/103 - 52s - 506ms/step - accuracy: 0.9599 - loss: 0.2496 - val_accuracy: 0.9519 - val_loss: 0.2495
Epoch 9/20
103/103 - 53s - 5